In [1]:
from datasets import load_dataset

# Load the JSONL file
data = load_dataset('json', data_files='instruction_training.jsonl')

# Check the data format
print(data['train'][0])  # Example entry


{'instruction': 'Begin the interview.', 'input': "The Clinician's Name is Brittany Quinn, and the date of the appointment is August 15", 'output': 'Good day, I am Dr. Brittany Quinn’s Interview Assistant and I was asked to connect with you before your appointment on August 15 at the medical clinic. My goal is to gather some background information for the doctor. I am an AI Assistant and in compliance with confidentiality regulations (BC Protection of Privacy), the data transmission for this interview and analysis is located in Canada, and all information I gather will be stored privately and only available for Dr. Brittany Quinn to review. Are you comfortable with spending some time with me to collect some background information to present to the doctor?'}


In [2]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 194
    })
})

In [3]:
def formatting_func(example):
  if example.get("input", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['input']}\n\n"
      f"### Response: \n"
      f"{example['output']}")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response:\n"
      f"{example['output']}")

  return {"text" : input_prompt}

In [4]:
formatted_dataset = data.map(formatting_func)

In [5]:
formatted_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 194
    })
})

In [6]:
formatted_dataset["train"][2]["text"]

"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nAsk for the patient's preferred name.\n\n### Response:\nNow, could you please tell me what name you prefer to be called by?"

In [7]:
import torch
import transformers
from peft import LoraConfig
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

model_id = "meta-llama/Llama-3.1-8B"

qlora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
)

RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):
[WinError 193] %1 is not a valid Win32 application

In [ ]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")

# def preprocess_function(examples):
#     return tokenizer(
#         examples['instruction'] + "\n" + examples['input'],
#         text_pair=examples['output'],
#         truncation=True,
#         padding='max_length',
#         max_length=2042,
#     )

# tokenized_data = data.map(preprocess_function, batched=True)